<a href="https://colab.research.google.com/github/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/blob/master/Stocks_Sharpe_Ratios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!pip install bokeh

from bokeh.io import curdoc

#bokeh.sampledata.download()
#from bokeh.sampledata.stocks import AAPL

# Perform necessary imports
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Band
from bokeh.models import NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

In [0]:
# Reading in the data
stock_data = pd.read_csv('https://raw.githubusercontent.com/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/master/stock_data.csv',
                         parse_dates=['Date'],
                         index_col='Date').dropna()

benchmark_data = pd.read_csv('https://raw.githubusercontent.com/PedroGuimaraesFerreira/Stocks-Sharpe-Ratios/master/benchmark_data.csv',
                             parse_dates=['Date'],
                             index_col='Date').dropna()


stock_returns = stock_data.pct_change()

sp_returns = benchmark_data['S&P 500'].pct_change()




In [0]:
def bollinger_bands(series, window=20, stds=2):
    
    #rolling metrics
    sma = series.rolling(window).mean()
    std = series.rolling(window).std()

    #bounds
    upper = sma + std * stds
    lower = sma - std * stds

    #yields
    return pd.DataFrame(index=series.index, data={
        'upper': upper,
        'mid': sma,
        'lower': lower
    })


In [0]:
 df_amazon = bollinger_bands(stock_data.Amazon)
 df_facebook = bollinger_bands(stock_data.Facebook)
 df_snp = bollinger_bands(benchmark_data['S&P 500'])

 df_amazon_er = bollinger_bands(excess_returns.Amazon)
 df_facebook_er = bollinger_bands(excess_returns.Facebook)

In [0]:
stock_data.info()
display(stock_data.shape)
display(stock_data.head())
display(stock_data.describe())

In [0]:
def datetime(x):
    return np.array(x, dtype=np.datetime64)

source = ColumnDataSource(data={
    'x' : datetime(stock_data.index),
    #amazon data
    'Amazon' : stock_data.Amazon,
    'Amazon_l' : df_amazon.lower,
    'Amazon_m' : df_amazon.mid,
    'Amazon_u' : df_amazon.upper,
    'Amazon_r' : stock_returns.Amazon,
    #amazon excess returns
    'A_er' : excess_returns.Amazon,
    'A_er_l': df_amazon_er.lower,
    'A_er_m': df_amazon_er.mid,
    'A_er_u': df_amazon_er.upper,

    #facebook data
    'Facebook' : stock_data.Facebook,
    'Facebook_l' : df_facebook.lower,
    'Facebook_m' : df_facebook.mid,
    'Facebook_u' : df_facebook.upper,
    'Facebook_r' : stock_returns.Facebook,
    #facebook excess returns
    'F_er' : excess_returns.Facebook,
    'F_er_l': df_facebook_er.lower,
    'F_er_m': df_facebook_er.mid,
    'F_er_u': df_facebook_er.upper,
})

source_snp = ColumnDataSource(data={
    'x' : datetime(benchmark_data.index),
    #S&P 500 data
    'y' : benchmark_data['S&P 500'],
    'l' : df_snp.lower,
    'm' : df_snp.mid,
    'u' : df_snp.upper,
    'r' : sp_returns
})

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

### Amazon

>#### Stock

In [0]:
p1 = figure(x_axis_type='datetime',
           title='Amazon Stock Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700,
           )
p1.axis.minor_tick_line_color = None
p1.yaxis.formatter = NumeralTickFormatter(format="$0.00")

# change just some things about the x-grid
p1.xgrid.grid_line_color = None

# change just some things about the y-grid
p1.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p1.background_fill_color="white"

#Amazon line plot
p1.line('x', 'Amazon', color='#FF9900', line_width=2, source=source)
r = p1.line('x', 'Amazon_m', color='black', source=source, line_alpha=0.3)

#glyph = r.glyph
#glyph.line_dash = [6, 1]


cr1 = p1.circle('x', 'Amazon', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

cr1_l = p1.circle('x', 'Amazon_l', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

cr1_m = p1.circle('x', 'Amazon_m', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

cr1_u = p1.circle('x', 'Amazon_u', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")


band1 = Band(base='x', lower='Amazon_l', upper='Amazon_u',
             source=source,
             fill_alpha=0.08, fill_color='black',
             line_width=1, line_color='black',
             level='underlay')

p1.add_layout(band1)

p1.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'            ),
        ( 'Actual Value',  '$@{Amazon}{0.000}' ),
        ( 'Bollinger Lower Value',  '$@{Amazon_l}{0.000}' ),
        ( '20 day Rolling Avg',  '$@{Amazon_m}{0.000}'),
        ( 'Bollinger Upper Value',  '$@{Amazon_u}{0.000}' )

    ],
    
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr1], 
    #show_arrow=True
))

output_notebook()
#show(p1)


> #### Daily %

In [285]:
r1 = figure(x_axis_type='datetime',
           title='Amazon Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r1.yaxis.formatter = NumeralTickFormatter(format="0.00%")
r1.axis.minor_tick_line_color = None
# change just some things about the x-grid
r1.xgrid.grid_line_color = None

# change just some things about the y-grid
r1.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

r1.background_fill_color="white"

#bands
band_1r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.1, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - 2*stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + 2*stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.067, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=stock_returns.Amazon.mean() - 3*stock_returns.Amazon.std(),
               upper=stock_returns.Amazon.mean() + 3*stock_returns.Amazon.std(),
               source=source,
               fill_alpha=0.035, fill_color='black',
               line_width=1, line_color='black',
               level='underlay')

r1.add_layout(band_3r1)

#Amazon line plot
r1.line('x', 'Amazon_r', color='#FF9900', line_width=3, source=source)

cr_r1 = r1.circle('x', 'Amazon_r', source=source, size=2,
              fill_color=None, hover_fill_color='black',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="black")

r1.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'),
        ( '% Change',  '@{Amazon_r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r1], 
    #show_arrow=True
))

output_notebook()
show(r1)

### Facebook

> #### Stock

In [0]:
p2 = figure(x_axis_type='datetime',
           title='Facebook Stock Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
p2.yaxis.formatter = NumeralTickFormatter(format="$0.00")
p2.axis.minor_tick_line_color = None
# change just some things about the x-grid
p2.xgrid.grid_line_color = None

# change just some things about the y-grid
p2.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p2.background_fill_color="white"

p2.border_fill_color = "white"
p2.min_border_left = 80

band2 = Band(base='x', lower='Facebook_l', upper='Facebook_u',
             source=source,
             fill_alpha=0.3, fill_color='#6699ff',
             line_width=1, line_color='#6699ff',
             level='underlay')

p2.add_layout(band2)

#Facebook line plot
p2.line('x', 'Facebook', color='#4267B2', line_width=2, source=source)
p2.line('x', 'Facebook_m', color='#6699ff', source=source, line_alpha=0.8)

cr2 = p2.circle('x', 'Facebook', source=source, size=2,
              fill_color="grey", hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

cr2_l = p2.circle('x', 'Facebook_l', source=source, size=2,
              fill_color="grey", hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

cr2_m = p2.circle('x', 'Facebook_m', source=source, size=2,
              fill_color="grey", hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

cr2_u = p2.circle('x', 'Facebook_u', source=source, size=2,
              fill_color="grey", hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')





#p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='vline'))

p2.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'            ),
        ( 'Actual Value',  '$@{Facebook}{0.000}'),
        ( 'Bollinger Lower Value',  '$@{Facebook_l}{0.000}' ),
        ( '20 day Rolling Avg',  '$@{Facebook_m}{0.000}'),
        ( 'Bollinger Upper Value',  '$@{Facebook_u}{0.000}' )
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr2], 
    #show_arrow=True
))


output_notebook()
#show(p2)





> #### Daily %

In [0]:
r2 = figure(x_axis_type='datetime',
           title='Facebook Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r2.yaxis.formatter = NumeralTickFormatter(format="0.00%")
r2.axis.minor_tick_line_color = None
# change just some things about the x-grid
r2.xgrid.grid_line_color = None

# change just some things about the y-grid
r2.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

r2.background_fill_color="white"

#bands
band_1r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.1, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - 2*stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + 2*stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.067, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=stock_returns.Facebook.mean() - 3*stock_returns.Facebook.std(),
               upper=stock_returns.Facebook.mean() + 3*stock_returns.Facebook.std(),
               source=source,
               fill_alpha=0.035, fill_color='#6699ff',
               line_width=1, line_color='#6699ff',
               level='underlay')

r2.add_layout(band_3r1)


r2.line('x', 'Facebook_r', color='#4267B2', line_width=3, source=source)

cr_r2 = r2.circle('x', 'Facebook_r', source=source, size=3,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#8b9dc3')

r2.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'            ),
        ( '% Change',  '@{Facebook_r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r2], 
    #show_arrow=True
))

output_notebook()
show(r2)


### S&P 500

> #### Stock

In [0]:
p3 = figure(x_axis_type='datetime',
           title='S&P 500 Data', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
p3.yaxis.formatter = NumeralTickFormatter(format="$0.00")
p3.axis.minor_tick_line_color = None
# change just some things about the x-grid
p3.xgrid.grid_line_color = None

# change just some things about the y-grid
p3.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

p3.background_fill_color="white"

p3.border_fill_color = "white"
p3.min_border_left = 80

band3 = Band(base='x', lower='l', upper='u',
             source=source_snp,
             fill_alpha=0.3, fill_color='#b3b3ff',
             line_width=1, line_color='#b3b3ff',
             level='underlay')

p3.add_layout(band3)


p3.line('x', 'y', color='#8080ff', line_width=2, source=source_snp)
p3.line('x', 'm', color='#b3b3ff', source=source_snp, line_alpha=0.8)

cr3 = p3.circle('x', 'y', source=source_snp, size=2,
              fill_color="grey", hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

cr3_l = p3.circle('x', 'l', source=source_snp, size=2,
              fill_color="grey", hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

cr3_m = p3.circle('x', 'm', source=source_snp, size=2,
              fill_color="grey", hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

cr3_u = p3.circle('x', 'u', source=source_snp, size=2,
              fill_color="grey", hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

#p.add_tools(HoverTool(tooltips=None, renderers=[cr], mode='vline'))

p3.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'            ),
        ( 'Actual Value',  '$@{y}{0.000}' ),
        ( 'Bollinger Lower Value',  '$@{l}{0.000}' ),
        ( '20 day Rolling Avg',  '$@{m}{0.000}'),
        ( 'Bollinger Upper Value',  '$@{u}{0.000}' ) 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr3], 
    #show_arrow=True
))


output_notebook()
#show(p3)




> #### Daily %

In [289]:
r3 = figure(x_axis_type='datetime',
           title='S&P 500 Daily % Returns', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700
           )
r3.yaxis.formatter = NumeralTickFormatter(format="0.00%")
r3.axis.minor_tick_line_color = None
# change just some things about the x-grid
r3.xgrid.grid_line_color = None

# change just some things about the y-grid
r3.ygrid.grid_line_alpha = 0.5

#p.ygrid.grid_line_dash = [6, 4]
r3.background_fill_color="white"

#4d4dff

#bands
band_1r1 = Band(base='x',
               lower=sp_returns.mean() - sp_returns.std(),
               upper=sp_returns.mean() + sp_returns.std(),
               source=source_snp,
               fill_alpha=0.08, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_1r1)

band_2r1 = Band(base='x',
               lower=sp_returns.mean() - 2*sp_returns.std(),
               upper=sp_returns.mean() + 2*sp_returns.std(),
               source=source_snp,
               fill_alpha=0.06, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_2r1)

band_3r1 = Band(base='x',
               lower=sp_returns.mean() - 3*sp_returns.std(),
               upper=sp_returns.mean() + 3*sp_returns.std(),
               source=source_snp,
               fill_alpha=0.05, fill_color='#4d4dff',
               line_width=1, line_color='#4d4dff',
               level='underlay')

r3.add_layout(band_3r1)


r3.line('x', 'r', color='#8080ff', line_width=3, source=source_snp)

cr_r3 = r3.circle('x', 'r', source=source_snp, size=3,
              fill_color=None, hover_fill_color='#0000ff',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color='#0000ff')

r3.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'        ),
        ( '% Change',  '@{r}{0.00%}') 
    ],
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_r3], 
    #show_arrow=True
))

output_notebook()
show(r3)

### Assembling grid plot

In [0]:
tab_1 = gridplot([[p1, r1], [p2, r2], [p3, r3]], plot_width=650, plot_height=280)  # open a browser

In [291]:
from bokeh.models.widgets import Panel
from bokeh.models.widgets import Tabs
from bokeh.plotting import figure, output_file, save

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

tab1 = Panel(child=tab_1, title='Stocks Data (20 day Bollinger Bands)')

layout = Tabs(tabs=[tab1])

# Specify the name of the output_file and show the result
output_file('tabs.html', mode='inline')
show(layout)



<IPython.core.display.Javascript object>

In [0]:
excess_returns = stock_returns.sub(sp_returns, axis=0)

In [0]:
from bokeh.models.widgets import Panel
from bokeh.models.widgets import Tabs
from bokeh.plotting import figure, output_file, save

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

tab2 = Panel(child=tab_2, title='Stocks Data (20 day Bollinger Bands)')

#layout = Tabs(tabs=[tab1])

# Specify the name of the output_file and show the result
output_file('tabs.html', mode='inline')
show(layout)


In [0]:
# calculate the mean of excess_returns 
avg_excess_return = excess_returns.mean()

# calculate the standard deviations
sd_excess_return = excess_returns.std()

In [397]:
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure

output_file("colormapped_bars.html")

stocks = list(avg_excess_return.index)
counts = list(avg_excess_return.values)
colors = ['#FF9900', '#4267B2']

source = ColumnDataSource(data=dict(stocks=stocks, counts=counts, color=colors))

means = figure(x_range=stocks, plot_height=250, title="Mean of the % Return Difference",
           toolbar_location=None, tools="")

means.vbar(x='stocks', top='counts', width=0.9, color='color', source=source, fill_alpha=0.3)

means.yaxis.formatter = NumeralTickFormatter(format="0.00%")
means.axis.minor_tick_line_color = None
means.xgrid.grid_line_color = None
means.ygrid.grid_line_color = None


show(means)




counts = list(sd_excess_return.values)
source = ColumnDataSource(data=dict(stocks=stocks, counts=counts, color=colors))

stds = figure(x_range=stocks, plot_height=250, title="Standard Deviation of the % Return Difference",
           toolbar_location=None, tools="")

stds.vbar(x='stocks', top='counts', width=0.9, color='color', source=source, fill_alpha=0.5)

stds.yaxis.formatter = NumeralTickFormatter(format="0.00%")
stds.axis.minor_tick_line_color = None
stds.xgrid.grid_line_color = None
stds.ygrid.grid_line_color = None


show(stds)



counts = list(annual_sharpe_ratio.values)
source = ColumnDataSource(data=dict(stocks=stocks, counts=counts, color=colors))

sharpe = figure(x_range=stocks, plot_height=250, title="Annualized Sharpe Ratio: Stocks vs S&P 500",
           toolbar_location=None, tools="")

sharpe.vbar(x='stocks', top='counts', width=0.9, color='color', source=source, fill_alpha=0.8)

sharpe.yaxis.formatter = NumeralTickFormatter(format="0.00%")
sharpe.axis.minor_tick_line_color = None
sharpe.xgrid.grid_line_color = None
sharpe.ygrid.grid_line_color = None


show(sharpe)




In [0]:
# calculate the daily sharpe ratio
daily_sharp_ratio = avg_excess_return.div(sd_excess_return)

# annualize the sharpe ratio
annual_factor = 252**(0.5)
annual_sharpe_ratio = daily_sharp_ratio.mul(annual_factor)

In [0]:
df = pd.DataFrame(pd.concat([avg_excess_return, sd_excess_return, annual_sharpe_ratio], axis=1), columns=['mean', 'std', 'annual sharpe ratio'])

In [0]:
# calculate the mean of excess_returns 
avg_excess_return = excess_returns.mean()

# calculate the standard deviations
sd_excess_return = excess_returns.std()

# calculate the daily sharpe ratio
daily_sharp_ratio = avg_excess_return.div(sd_excess_return)

# annualize the sharpe ratio
annual_factor = 252**(0.5)
annual_sharpe_ratio = daily_sharp_ratio.mul(annual_factor)


df = pd.concat([avg_excess_return, sd_excess_return, annual_sharpe_ratio], axis=1)
df.columns = ['mean', 'std', 'annual sharpe ratio']

In [378]:
er = figure(x_axis_type='datetime',
           title='Stocks % Excess Returns vs. S&P 500', 
           #x_axis_label='Date',
           #y_axis_label='Value ($)',
           plot_height=400,
           plot_width=700,
           )
er.axis.minor_tick_line_color = None
er.yaxis.formatter = NumeralTickFormatter(format="0%")

# change just some things about the x-grid
er.xgrid.grid_line_color = None

# change just some things about the y-grid
er.ygrid.grid_line_alpha = 0.5
#p.ygrid.grid_line_dash = [6, 4]

er.background_fill_color="white"

#Amazon line plot
er.line('x', 'A_er', color='#FF9900', line_width=2, source=source)
band_a = Band(base='x', lower='A_er_l', upper='A_er_u',
             source=source,
             fill_alpha=0.3, fill_color='#FF9900',
             line_width=2, line_color='#FF9900',
             level='underlay')

cr_a = er.circle('x', 'A_er', source=source, size=2,
              fill_color=None, hover_fill_color='#FF9900',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#FF9900")

cr1_al = er.circle('x', 'A_er_l', source=source, size=2,
              fill_color=None, hover_fill_color='#FF9900',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#FF9900")

cr1_am = er.circle('x', 'A_er_m', source=source, size=2,
              fill_color=None, hover_fill_color='#FF9900',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#FF9900")

cr1_au = er.circle('x', 'A_er_u', source=source, size=2,
              fill_color=None, hover_fill_color='#FF9900',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#FF9900")

er.add_layout(band_a)



#Facebook line plot
er.line('x', 'F_er', color='#4267B2', line_width=2, source=source)

band_f = Band(base='x', lower='F_er_l', upper='F_er_u',
             source=source,
             fill_alpha=0.3, fill_color='#8b9dc3',
             line_width=2, line_color='#8b9dc3',
             level='underlay')

cr_f = er.circle('x', 'F_er', source=source, size=2,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#8b9dc3")

cr1_fl = er.circle('x', 'F_er_l', source=source, size=2,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#8b9dc3")

cr1_fm = er.circle('x', 'F_er_m', source=source, size=2,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#8b9dc3")

cr1_fu = er.circle('x', 'F_er_u', source=source, size=2,
              fill_color=None, hover_fill_color='#8b9dc3',
              fill_alpha=0.05, hover_alpha=1,
              line_color=None, hover_line_color="#8b9dc3")



er.add_tools(HoverTool(
    tooltips=[
        ( 'Date',   '@x{%F}'            ),
        ( 'Amazon Excess % Return vs. S&P 500',  '@{A_er}{0.000%}' ),
        ( 'Facebook Excess % Return vs. S&P 500',  '@{F_er}{0.000%}' )

    ],
    
    formatters={
        'x'      : 'datetime', # use 'datetime' formatter for 'date' field
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[cr_f], 
    #show_arrow=True
))

er.add_layout(band_f)

output_notebook()
show(er)


In [0]:
tab_2 = gridplot([[er], [means, stds, sharpe]], sizing_mode="scale_both")  # open a browser

In [399]:
tab2 = Panel(child=tab_2, title='Sharpe Ratio Analysis')

layout = Tabs(tabs=[tab1, tab2])

# Specify the name of the output_file and show the result
output_file('tabs.html', mode='inline')
show(layout)